Run this notebook to

- Merge all feedback epochs from all participants and all sessions and save merged epochs file
- Plot FRN for correct vs incorrect trials
- Plot FRN for correct vs incorrect trials split by feedback condition
- Plot FRN for lower than median vs higher than median confidence 
- Plot FRN for lower than median vs higher than median confidence split by feedback condition
- Plot all of the above separately for the strategic and observational conditions


## Import stuff

In [ ]:
import os
import numpy as np
import mne
from mne.stats import permutation_t_test
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr 

million = 1000000.


#%matplotlib qt
%matplotlib inline

input_dir = 'TaskfeedbackEpochsMastoids'


In [ ]:
def load_subj_eeg(path, file, downsample=None):
    fp = os.path.join(path, '%s-epo.fif' % file)
    print('>>> Loading %s' % fp)
    epochs = mne.read_epochs(fp, preload=True)
    if downsample is not None:
        epochs = epochs.resample(downsample)
    return epochs

def load_all_eeg(path, files, downsample=None):
    subject_epochs = [load_subj_eeg(path, file, downsample=downsample) for file in files]
    epochs = mne.epochs.concatenate_epochs(subject_epochs)
    return epochs

## Merge epochs

In [ ]:
#participant_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]

sessions = [1, 2]

participant_files = []

for session in sessions:
    
    # only strategic condition has feedback
    if session == 1:
        participant_numbers = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
    if session == 2:
        participant_numbers = [2, 4, 6, 8, 10, 12, 16, 18, 20, 21]
        
    for subject in participant_numbers:
        participant_files.append('%i_%i' % (subject, session))

In [ ]:
remerge = True
if remerge:
    feedback_epochs = load_all_eeg(path='%s/' % input_dir, files=participant_files)
    feedback_epochs.save('mergedData/feedback_epoch_mastoids-epo.fif', overwrite=True)
else:
    feedback_epochs = mne.read_epochs('mergedData/feedback_epoch_mastoids-epo.fif')
    
feedback_epochs = feedback_epochs.pick_types(eeg=True)
data = feedback_epochs.metadata

## correct vs incorrect grand average (first calculating each participant's average)

In [ ]:
grand_average_correct = []
grand_average_incorrect = []

correct_epochs = feedback_epochs['joint_correct == True']
incorrect_epochs = feedback_epochs['joint_correct == False']

for subject in participant_numbers:
    grand_average_correct.append(correct_epochs['participant == %i' % subject].average())
    grand_average_incorrect.append(incorrect_epochs['participant == %i' % subject].average())


grand_average_correct = mne.grand_average(grand_average_correct)
grand_average_incorrect = mne.grand_average(grand_average_incorrect)

evokeds = dict(correct=grand_average_correct,
               incorrect=grand_average_incorrect)

mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)


## correct vs incorrect split by partner grand average 


In [ ]:
grand_average_underconf_correct = []
grand_average_underconf_incorrect = []
grand_average_overconf_correct = []
grand_average_overconf_incorrect = []

underconf_epochs = feedback_epochs['partner == "underconfident"']
underconf_correct_epochs = underconf_epochs['joint_correct == True']
underconf_incorrect_epochs = underconf_epochs['joint_correct == False']

overconf_epochs = feedback_epochs['partner == "overconfident"']
overconf_correct_epochs = overconf_epochs['joint_correct == True']
overconf_incorrect_epochs = overconf_epochs['joint_correct == False']



for subject in participant_numbers:    
    grand_average_underconf_correct.append(underconf_correct_epochs['participant == %i' % subject].average())
    grand_average_underconf_incorrect.append(underconf_incorrect_epochs['participant == %i' % subject].average())
    grand_average_overconf_correct.append(overconf_correct_epochs['participant == %i' % subject].average())
    grand_average_overconf_incorrect.append(overconf_incorrect_epochs['participant == %i' % subject].average())

grand_average_underconf_correct = mne.grand_average(grand_average_underconf_correct)
grand_average_underconf_incorrect = mne.grand_average(grand_average_underconf_incorrect)
grand_average_overconf_correct = mne.grand_average(grand_average_overconf_correct)
grand_average_overconf_incorrect = mne.grand_average(grand_average_overconf_incorrect)


evokeds = dict(underconf_incorrect=grand_average_underconf_incorrect,
               underconf_correct=grand_average_underconf_correct,
               overconf_incorrect=grand_average_overconf_incorrect,
               overconf_correct=grand_average_overconf_correct)

mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)


## Low vs high confidence grand average 

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
feedback_epochs.metadata['confidence_z_by_participant'] = feedback_epochs.metadata['participant_confidence'].groupby(feedback_epochs.metadata['participant']).transform(zscore)
        
median_confidence = feedback_epochs.metadata['confidence_z_by_participant'].median()


In [ ]:
grand_average_high_conf = []
grand_average_low_conf = []

high_conf_epochs = feedback_epochs['confidence_z_by_participant > 0']
low_conf_epochs = feedback_epochs['confidence_z_by_participant <= 0']


        
for subject in participant_numbers:
    grand_average_high_conf.append(high_conf_epochs['participant == %i' % subject].average())
    grand_average_low_conf.append(low_conf_epochs['participant == %i' % subject].average())


grand_average_high_conf = mne.grand_average(grand_average_high_conf)
grand_average_low_conf = mne.grand_average(grand_average_low_conf)

evokeds = dict(high_confidence=grand_average_high_conf,
               low_confidence=grand_average_low_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)


## Low vs high confidence split by partner grand average 


In [ ]:
grand_average_underconf_high_conf = []
grand_average_underconf_low_conf = []
grand_average_overconf_high_conf = []
grand_average_overconf_low_conf = []

underconf_epochs = feedback_epochs['partner == "underconfident"']
underconf_high_conf_epochs = underconf_epochs['confidence_z_by_participant > 0']
underconf_low_conf_epochs = underconf_epochs['confidence_z_by_participant <= 0']

overconf_epochs = feedback_epochs['partner == "overconfident"']
overconf_high_conf_epochs = overconf_epochs['confidence_z_by_participant > 0']
overconf_low_conf_epochs = overconf_epochs['confidence_z_by_participant <= 0']



for subject in participant_numbers:    
    grand_average_underconf_high_conf.append(underconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_underconf_low_conf.append(underconf_low_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_high_conf.append(overconf_high_conf_epochs['participant == %i' % subject].average())
    grand_average_overconf_low_conf.append(overconf_low_conf_epochs['participant == %i' % subject].average())

grand_average_underconf_high_conf = mne.grand_average(grand_average_underconf_high_conf)
grand_average_underconf_low_conf = mne.grand_average(grand_average_underconf_low_conf)
grand_average_overconf_high_conf = mne.grand_average(grand_average_overconf_high_conf)
grand_average_overconf_low_conf = mne.grand_average(grand_average_overconf_low_conf)


evokeds = dict(underconf_low_confidence=grand_average_underconf_low_conf,
               underconf_high_confidence=grand_average_underconf_high_conf,
               overconf_low_confidence=grand_average_overconf_low_conf,
               overconf_high_confidence=grand_average_overconf_high_conf)

mne.viz.plot_compare_evokeds(evokeds, picks=['Cz'], invert_y=False)
